## Load Conversations

In [95]:
import sys
sys.path.append('.')

import pandas as pd
from utils import load_jsonl, dump_jsonl
# import numpy as np

In [96]:
def add_subject(data):
    random.shuffle(data)
    n = int(len(data)*0.5)

    p1 = pd.DataFrame(data[0:n])
    p2 = pd.DataFrame(data[n:])

    p1["label"] = p1["label_1"]
    p1["subject"] = 0

    p2["label"] = p2["label_2"]
    p2["subject"] = 1

    df = pd.concat([p1, p2]).sample(frac=1).reset_index(drop=True)
    
    return df
    
def get_task1_conver(in_dir, col_labels, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
    
    
    def to_message_str(messages, users):
        s = []
        u = []
        for m in messages:
            if users[m['user_id']] == "SYS":
                s.append(m['text'])
            else:
                u.append(m['text'])
                
        return s, u
        
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["date_created"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())==0 else "SYS"
                users[m["user_id"]] = username
                
#         if len(users)>2:
#             print("More than 1 users", len(users))
        
        
        messages = row["messages"]
        chunk_size = 100
        for i in range(0, len(messages), chunk_size):
            sub_messages = messages[i:i+chunk_size]
            s, u = to_message_str(sub_messages, users)
            
            
            o = {
                "user": u,
                "sys": s,
                "nturn": len(messages)
            }
            
            skip = False
            for i, col in enumerate(col_labels):
                l = row[col]
                if pd.isna(l) or (l in skips):
                    skip = True
                    break
                
                o["label_"+str(i+1)] = l
                

            if skip:
                continue

            newdata.append(o)

    
    if len(col_labels)==1:
        df = pd.DataFrame(newdata)
        df["label"] = df["label_1"]
        df["subject"] = 0
    else:
        df = add_subject(newdata)
        
    return df

In [97]:
df = get_task1_conver("./Task3/annotated/annotated.jsonl", ["authority_1", "authority_3"])
df.head()

Loaded 1221 records from ./Task3/annotated/annotated.jsonl


,user,sys,nturn,label_1,label_2,label,subject
0,"[มาแล้วคะ, ชอบออกกำลังกายด้วยเหรอ, ปุ้ยนี่ชอบม...","[ค่ะ, ปกติออกกำลังกายไหมคะ, ไม่ค่อยได้ออกกำบัง...",35,2. Normal,2. Normal,2. Normal,0
1,"[วันนี้ฝนตกมั้ยคะ, ที่นี่ร้อนมากเลยค่ะ, ดีจังค...","[ไม่ค่ะ, เช่นกันค่ะ, แต่แถวบ้านยังไม่ร้อนมากค่...",38,1. Respect,2. Normal,1. Respect,0
2,"[สวัสดีค่ะ, วันนี้ก็พบกับรายการของเราอีกเช่นเค...","[เกินไปๆ, ก็เคนะ แปลกใหม่ดี, ปกติเป็นพิธีกร พู...",35,2. Normal,1. Respect,1. Respect,1
3,"[เทอยื่นภาษีไปยังงงงงง, แล้วเทอได้ทวิ 50 ของออ...","[ยังๆ, แล้วมันต้องใช้อะไรบ้าง, เค้าไม่เคยยื่น,...",39,2. Normal,2. Normal,2. Normal,1
4,"[เราอยู่ในช่วง, เพื่อนไม่มี, 5555555555, วันนี...","[สปอนเซอร์ไม่มา, ถถถถถถถถถถถถ, เรามีแผน, เอาละ...",35,2. Normal,3. Not respect,2. Normal,0


In [98]:
df.groupby("label").count()

,user,sys,nturn,label_1,label_2,subject
label,,,,,,
1. Respect,273,273,273,273,273,273
2. Normal,822,822,822,822,822,822
3. Not respect,114,114,114,114,114,114


In [99]:
# df.groupby("label").count()

In [140]:
import random
def get_task2_conver(in_dir, col_labels, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
    
    n=0
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["created_at"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())!=0 else "SYS"
                users[m["user_id"]] = username
                
        if len(users)>2:
            print("More than 1 users", len(users))
        
        
        
        messages = row["messages"]
        s = []
        u = []
        for m in messages:
            text = m['text'].replace("[USR]", "").replace("[URL]", "URL")
            if users[m['user_id']] == "SYS":
                s.append(text)
            else:
                u.append(text)
    
        labels = [row[col] for col in col_labels]
        

        skip = False
        for l in labels:
            # if (l in skips):
            if pd.isna(l) or (l in skips):
                skip = True
                break
        
        
        if skip:
            n+=1
            continue
            
        o = {
            "user": u,
            "sys": s,
            "nturn": len(messages)
        }
        
        for i, l in enumerate(labels):
            o["label_"+str(i+1)] = l
            
        newdata.append(o)
    
    if len(col_labels)==1:
        df = pd.DataFrame(newdata)
        df["label"] = df["label_1"]
        df["subject"] = 0
        return df
    else:
        df = add_subject(newdata)
        
    return df

# df = get_task2_conver("./Task2/annotated/annotated.jsonl", ["authority_1", "authority_3"])
# # df.groupby("label").count()
# df

In [143]:
df = get_task2_conver("./Task2/annotated/annotated.jsonl", ["authority_1",  "authority_2"])
df.head()

Loaded 2463 records from ./Task2/annotated/annotated.jsonl


,user,sys,nturn,label_1,label_2,label,subject
0,[ Bio jelly night 🌙🌃\nไม่ใช่ยานอนหลับแต่เป็นอา...,[ตื่นมาก็งงเลอะ #คลับเฮ้าส์toxic URL],3,1. Respect,2. Normal,2. Normal,1
1,[ ขอซองสีชมพูนะคะ😆😍😍💌],[ว้อยยยยย!!! ไม่ได้แจกการ์ด จะแจก #ยาดมน้องแบม...,3,1. Respect,2. Normal,2. Normal,1
2,[ อยากดูคะ],[ฟังแล้วหดหู่​คนที่อยากทำงานเพื่อปชช.ต้องเจออะ...,3,2. Normal,2. Normal,2. Normal,0
3,[ ถ้ามีการระบาดเกิดขึ้น มันจะต้องโทษว่าเป็นเพร...,[แบน 8 ชาติจากอาฟริกา แต่โอไมครอนดันมาจากสเปน ...,3,2. Normal,3. Not respect,3. Not respect,1
4,[ บ้านเรา มีแต่คนนอกพื้นที่ไปทำบุญ 🤣เราว่าพอก...,[โอเค 👍🏻 ความคิดใครความคิดมัน อย่ามองคน ที่ภ...,10,3. Not respect,2. Normal,2. Normal,1


In [145]:
# df.count()

## Load Lexicons

In [53]:
from pythainlp.tokenize import word_tokenize

In [54]:
import json
with open("./PrivateSpace/thai-dictionary/RoyalInstituteDictionary/words.json", encoding="utf-8") as fin:
    raw = json.load(fin)
    thaidict_royal = set()
    for k in raw:
        thaidict_royal.update(raw[k])

In [55]:
lexicons_arr = load_jsonl("lexicons.jsonl")

Loaded 25573 records from lexicons.jsonl


In [56]:
from collections import defaultdict
tags = set()
lexicons = {}
lexicons_keys = defaultdict(list)

for key, values  in lexicons_arr:
    if len(key) <= 1:
        continue
        
    key = key.lower()
    if key.endswith("rep"):
        key = key.replace("rep", "")
        
    w = word_tokenize(key)
    
    lexicons_keys[w[0]].append(key)
    
    tag = [t for t in values["tags"] if not t.startswith("cat:")]
    lexicons[key] = tag
    tags.update(tag)

In [57]:
from pythainlp.util import countthai

In [58]:
import re
from itertools import groupby
import emoji
from pythainlp.util import countthai
from tqdm import tqdm

def rm_reptitive(text):
    s = ""
    groups = groupby(text)
    for label, group in groups:
        g = list(group)
        if len(g) >= 3:
            s += f"{label*3} rep "
        else:
            s += "".join(g)
    return s

def remove_space(sent):
    newwords = []
    for w in sent:
        if len(w.strip())==0:
            continue
        newwords.append(w)
    return newwords
    
def analyse_conv_per_person(texts):
    
    # Word Statistic
    texts = [t.lower() for t in texts]
    texts = [rm_reptitive(t) for t in texts]
    words = [word_tokenize(t) for t in texts]
    words = [remove_space(w) for w in words]
    
    nlongword = 0
    ndict = 0
    for sent in words:
        ndict += sum([1 if w in thaidict_royal else 0 for w in sent])
        nlongword += sum([1 if len(w) > 7 else 0 for w in sent])
        nthai = sum([1 if countthai(w) > 50 else 0 for w in sent])
        
        
    
    uwords = set()
    for sent in words:
        uwords.update(sent)
    
    # Lexicon 
    lex = []
    for sidx, sent in enumerate(words):
        for widx, w in enumerate(sent):
            if w not in lexicons_keys:
                continue 
            
            s = "".join(sent[widx:])
            for l in lexicons_keys[w]:
                if not s.startswith(l):
                    continue

                lex.extend(lexicons[l])
#                 print(">>", w, l, lexicons[l])

    lexcat = {}
    for l in lex:
        if l not in lexcat:
            lexcat[l] = 0
        lexcat[l] += 1
    
    # Stylistic words
    nrepeat = 0
    for sidx, sent in enumerate(words):
        nrepeat += sum([1 if w=="rep" else 0 for w in sent])
    
    s = " ".join(texts)
    nemoji = emoji.emoji_count(s)
    

    return {
        "nsent": len(texts),
        "nword": sum([len(w) for w in words]),
        "ndict": ndict,
        "nunique": len(uwords),
        "nlongword": nlongword,
        "nrepeat": nrepeat,
        "nthai": nthai,
        "nemoji": nemoji,
        **lexcat
    }
  


def analyse_conversation(df):
    metrics = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        ms = analyse_conv_per_person(row["sys"])
        mu = analyse_conv_per_person(row["user"])
        metrics.append((ms, mu))
    
    return metrics

analyse_conv_per_person(["เมิงงงงงงงมันโง่เหมือนควายยยยยยยยย"]);

## Run Analysis

In [64]:
import pickle
def save_obj_values(filepath, obj):
  with open(filepath, 'wb') as fin:
    pickle.dump(obj, fin)
    
def load_obj_values(filepath):
  with open(filepath, 'rb') as fin:
    obj = pickle.load(fin)
  return obj

In [207]:
analysis_values = {
    "public": {
        "closeness": [],
        "authority": [],
    },
    "private": {
        "closeness": [],
        "authority": [],
    }
}

In [208]:
annotation_pairs = [
    ("closeness_1", "closeness_2"),
    ("closeness_1", "closeness_3"),
    ("closeness_2", "closeness_3"),
]
for col1, col2 in annotation_pairs:
    df = get_task2_conver("./Task2/annotated/annotated.jsonl", [col1, col2], skips = [])
    metrics = analyse_conversation(df)
    
    analysis_values["public"]["closeness"].append({
        "values": metrics,
        "labels": df["label"].values,
        "subject": df["subject"].values
    })
    
    
    df = get_task1_conver("./Task3/annotated/annotated.jsonl", [col1, col2], skips = [])
    metrics = analyse_conversation(df)
    
    analysis_values["private"]["closeness"].append({
        "values": metrics,
        "labels": df["label"].values,
        "subject": df["subject"].values
    })

Loaded 2463 records from ./Task2/annotated/annotated.jsonl


100%|██████████████████████████████████████| 1714/1714 [00:04<00:00, 409.57it/s]


Loaded 1221 records from ./Task3/annotated/annotated.jsonl


100%|██████████████████████████████████████| 1219/1219 [00:05<00:00, 233.29it/s]


Loaded 2463 records from ./Task2/annotated/annotated.jsonl


...

100%|██████████████████████████████████████| 1218/1218 [00:05<00:00, 223.20it/s]


Loaded 2463 records from ./Task2/annotated/annotated.jsonl


100%|██████████████████████████████████████| 1718/1718 [00:03<00:00, 445.44it/s]


Loaded 1221 records from ./Task3/annotated/annotated.jsonl


100%|██████████████████████████████████████| 1220/1220 [00:05<00:00, 237.17it/s]


In [209]:
annotation_pairs = [
    ("authority_1", "authority_2"),
    ("authority_1", "authority_3"),
    ("authority_2", "authority_3"),
]
for col1, col2 in annotation_pairs:
    df = get_task2_conver("./Task2/annotated/annotated.jsonl", [col1, col2], skips = [])
    metrics = analyse_conversation(df)
    
    analysis_values["public"]["authority"].append({
        "values": metrics,
        "labels": df["label"].values,
        "subject": df["subject"].values
    })
    
    
    df = get_task1_conver("./Task3/annotated/annotated.jsonl", [col1, col2], skips = [])
    metrics = analyse_conversation(df)
    
    analysis_values["private"]["authority"].append({
        "values": metrics,
        "labels": df["label"].values,
        "subject": df["subject"].values
    })

Loaded 2463 records from ./Task2/annotated/annotated.jsonl


100%|████████████████████████████████████████| 966/966 [00:02<00:00, 480.88it/s]


Loaded 1221 records from ./Task3/annotated/annotated.jsonl


100%|██████████████████████████████████████| 1209/1209 [00:05<00:00, 234.75it/s]


Loaded 2463 records from ./Task2/annotated/annotated.jsonl


...

100%|██████████████████████████████████████| 1209/1209 [00:04<00:00, 246.49it/s]


Loaded 2463 records from ./Task2/annotated/annotated.jsonl


100%|██████████████████████████████████████| 1638/1638 [00:03<00:00, 494.34it/s]


Loaded 1221 records from ./Task3/annotated/annotated.jsonl


100%|██████████████████████████████████████| 1221/1221 [00:04<00:00, 247.24it/s]


In [210]:
save_obj_values("Results/analysis_values_sub.pkl", analysis_values)

In [211]:
analysis_values = load_obj_values("Results/analysis_values_sub.pkl")

## Appendix: Regression Analysis

In [222]:
metric_names = {
    "Word Statistics" : {
        "nsent": "Number of utterance",
        "nword": "Number of word",
    },
    "Lexicon Diversity" : {
        "nunique": "Vocabulary size",
        "nthai": "Thai words",
        "nlongword": "Long words",
        "ndict": "Dictionary words"
    },
    
    "Function Words": {

        "pronoun": "Pronoun",
        "pronoun_1st": ">> 1st person pronoun",
        "pronoun_2nd": ">> 2nd person pronoun",
        "pronoun_3rd": ">> 3rd person pronoun",
        "pronoun_misspelling": ">> Pronoun in non-standard spelling",

        "particles": "Particles",
        "particles_SARP": ">> Socially-related particles",
        "particles_notSARP": ">> Non-socially-related particles",
        "particles_misspelling": ">> Particle in non-standard spelling",
    },
    
    "Sentiment-related": {
    
        "sentiment": "Sentiment words",
        "sentiment_positive": ">> Positive words",
        "sentiment_negative": ">> Negative words",
    },
    
    "Internet Lexicons": {
        "misspelling": "Spelling variation",
        "misspelling_common": ">> Common misspelt words",
        "misspelling_intention": ">> Sematic variation",
        "misspelling_shorten": ">> Simplified variation",
        "nrepeat": ">> Repeated characters",

        "abbr": "Abbreviation",
        # "slang": "Slang",
        "swear": "Swear words",
        "transliterated": "Transliteration",
    }
}

In [223]:
clse_print_labels = ['1. Close', '2. Know each other', "3. Don't know each other", "4. Don't like each other"]
auth_print_labels = ['0. Very respect', '1. Respect', '2. Normal', '3. Not respect']

In [224]:

    
def authority_to_vec1(label, subject):
    if label == '0. Very respect':
        return {"x1": 1, "x2": 0, "x3": subject, "x4": subject, "x5": 0}
    elif label == '1. Respect': 
        return {"x1": 0, "x2": 0, "x3": subject, "x4": 0, "x5": 0}
    elif label == '2. Normal':
        return {"x1": 0, "x2": 1, "x3": subject, "x4": 0, "x5": subject}
    elif label == '3. Not respect':
        return None
    else:
        return None

def closeness_to_vec1(label, subject):
    if label == "1. Close":
        return {"x1": 1, "x2": 0, "x3": subject, "x4": subject, "x5": 0}
    elif label == "2. Know each other":
        return {"x1": 0, "x2": 0, "x3": subject, "x4": 0, "x5": 0}
    elif label == "3. Don't know each other": 
        return {"x1": 0, "x2": 1, "x3": subject, "x4": 0, "x5": subject}
    elif label == "4. Don't like each other":
        return None
    else:
        return None

In [225]:

    
def to_features(analysis_values, setting, relation, idx, to_vec_func, skips=[]):
    exp = analysis_values[setting][relation][idx]
    values = exp["values"]
    labels = exp["labels"]
    subject = exp["subject"]

    assert(len(values)==len(labels))


    rows = []
    for (ms, mu), l, s in zip(values, labels, subject):
        if l in skips:
            continue

        x = to_vec_func(l, s)
        if x is None:
            continue

        for m in mu:
            if m in ["nsent", "nword"]:
                v = mu[m]
            else:
                v = mu[m]*100/mu["nword"]


            rows.append({
                "metric": m,
                "value": v,
                **x
            })

        if "particles_SARP" in mu:
            particles_notSARP = mu["particles"] - mu["particles_SARP"]
            rows.append({
                "metric": "particles_notSARP",
                "value": particles_notSARP,
                **x
            })
            
    feats = pd.DataFrame(rows)  
    return feats

In [244]:
import statsmodels.api as sm
import numpy as np

def print_anova_test(feats, n=3):
    feat_names = feats["metric"].unique()
    
    feat_important = []
    summary = []
    for f in feat_names:
        if "cat:" in f:
            continue

        try:
            d = feats[feats["metric"]==f]
        #     print(f, len(d))
            x_columns = [f"x{i+1}" for i in range(n)]
            X = d[x_columns]
            X = sm.add_constant(X)
            Y = d["value"]
            model = sm.OLS(Y,X)
            results = model.fit()
            
            ncoef = n + 1
            
            t_test = results.t_test(np.identity(ncoef))
            f_test = results.f_test(np.identity(ncoef))
            o = {"feat": f, "f_value": f_test.pvalue}
            for i in range(ncoef):
                o[f"coef{i}"] = t_test.effect[i]
                
            for i in range(ncoef):
                o[f"pval{i}"] = t_test.pvalue[i]
                
            feat_important.append(o)
            summary.append(results)
        except Exception as e:
            print("error", f, e)

    outputs = pd.DataFrame(feat_important)
    return outputs, summary

In [249]:
def print_weights(outputs):
    
    coefs = {}
    coef_labels = ["b", "w1", "w2", "w3", "w4", "w5"]
    for i, label in enumerate(coef_labels):
        for output in outputs:
            for _, row in output.iterrows():
                key = (label, row["feat"])
                if key not in coefs:
                    coefs[key] = (row[f"coef{i}"]/len(outputs), row[f"pval{i}"]/len(outputs))
                else: 
                    ## Average the coefs across experiments
                    _coef, _pval = coefs[key]
                    _coef += row[f"coef{i}"]/len(outputs)
                    _pval += row[f"pval{i}"]/len(outputs)
                    coefs[key] = (_coef, _pval)

#     print(coefs)

    printed_text = ""
    for g in metric_names:
        # print("\multicolumn{4}{l}{\\textit{"+g+"}} \\\\")
        printed_text += "\multicolumn{7}{l}{\\textit{"+g+"}} \\\\"+"\n"
        for m in metric_names[g]:
            s = f"{metric_names[g][m]} "
            for l in coef_labels:                    
                val, pval = coefs[(l, m)]
                if l=="b":
                    s += f"& {val:.2f} "
                elif pval < 0.05:
                    s += "& \cellcolor{gray!25} "+f"{val:.2f} "
                else:
                    s += f"& {val:.2f} "
            s += "\\\\"
            # print(s)
            printed_text += s+"\n"
        printed_text += "&  & &\\\\" + "\n"
        printed_text += "\hline" + "\n"
        # print("&  & &\\\\")
        # print("\hline")
    return printed_text

In [250]:
# printed_text1 = print_weights([outputs])
# print(printed_text1)

In [251]:
import warnings
warnings.filterwarnings("ignore")

outputs = []
for i in range(3):
    feats = to_features(analysis_values, "public", "closeness", i, closeness_to_vec1, skips=[])
    output, summary = print_anova_test(feats, n=5)
    outputs.append(output)
printed_text1 = print_weights(outputs)

outputs = []
for i in range(3):
    feats = to_features(analysis_values, "public", "authority", i, authority_to_vec1, skips=["3. Not respect"])
    output, summary = print_anova_test(feats, n=5)
    outputs.append(output)
printed_text2 = print_weights(outputs)

error particles_?? wrong shape for coefs
error particles_?? r_matrix performs f_test for using dimensions that are asymptotically non-normal
error particles_?? wrong shape for coefs
error swear r_matrix performs f_test for using dimensions that are asymptotically non-normal
error particles_?? wrong shape for coefs


In [252]:
# summary[0].summary()

In [253]:
def closeness_to_vec2(label, subject):
    if label == "1. Close":
        return {"x1": 1, "x2": 0, "x3": subject, "x4": subject, "x5": 0}
    elif label == "2. Know each other": 
        return {"x1": 0, "x2": 0, "x3": subject, "x4": 0, "x5": 0}
    elif label == "3. Don't know each other": ## Base category
        return {"x1": 0, "x2": 1, "x3": subject, "x4": 0, "x5": subject}
    elif label == "4. Don't like each other":
        return None
    else:
        return None
    
def authority_to_vec2(label, subject):
    if label == '0. Very respect':
        return None
    elif label == '1. Respect': 
        return {"x1": 1, "x2": 0, "x3": subject, "x4": subject, "x5": 0}
    elif label == '2. Normal': ## Base category
        return {"x1": 0, "x2": 0, "x3": subject, "x4": 0, "x5": 0}
    elif label == '3. Not respect':
        return {"x1": 0, "x2": 1, "x3": subject, "x4": 0, "x5": subject}
    else:
        return None

In [255]:
outputs = []
for i in range(3):
    feats = to_features(analysis_values, "private", "closeness", i, closeness_to_vec2, skips=[])
    output, summary = print_anova_test(feats, n=5)
    outputs.append(output)
printed_text3 = print_weights(outputs)

outputs = []
for i in range(3):
    feats = to_features(analysis_values, "private", "authority", i, authority_to_vec2, skips=[])
    output, summary = print_anova_test(feats, n=5)
    outputs.append(output)
printed_text4 = print_weights(outputs)

In [256]:
sections = [
    "Setting 2: Public Conversations with Labels from 3rd Party ",
    "Setting 3: Private Conversations with Labels from 3rd Party ",
]

table_contents = [
    (printed_text1, printed_text2),
    (printed_text3, printed_text4),
]

printed_text = ""
for section, (t1, t2) in zip(sections, table_contents):
    
    printed_text += "\subsection{"+section+"}"+"\n\n"
    
    printed_text += '''\subsubsection{Closeness}
\\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\\tabcolsep}|c|c|c|c|c|c|
}
    \hline

    Lexical Features & b & w1 & w2 & w3 & w4 & w5 \\\\
    \hline
    \endfirsthead

    \endhead
    '''
    
    printed_text += ("\n"+t1).replace("\n", "\n    ")
    printed_text += "\n"
    printed_text += "\end{longtable}"
    printed_text += "\n\n"

    printed_text += '''\clearpage
\subsubsection{Respect}
\\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\\tabcolsep}|c|c|c|c|c|c|
}
    \hline

    Lexical Features & b & w1 & w2 & w3 & w4 & w5 \\\\
    \hline
    \endfirsthead

    \endhead
    '''
    
    printed_text += ("\n"+t2).replace("\n", "\n    ")
    printed_text += "\n"
    printed_text += "\end{longtable}"
    printed_text += "\n\n"
    
    printed_text += "\clearpage"
    printed_text += "\n\n"
    # break
    
print(printed_text)

\subsection{Setting 2: Public Conversations with Labels from 3rd Party }

\subsubsection{Closeness}
\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\tabcolsep}|c|c|c|c|c|c|
}
    \hline

    Lexical Features & b & w1 & w2 & w3 & w4 & w5 \\
    \hline
    \endfirsthead

    \endhead
    
    \multicolumn{7}{l}{\textit{Word Statistics}} \\
    Number of utterance & 1.52 & -0.06 & 0.01 & 0.05 & -0.01 & -0.10 \\
    Number of word & 37.78 & \cellcolor{gray!25} -13.95 & -0.48 & 3.76 & -3.18 & -5.50 \\
    &  & &\\
    \hline
    \multicolumn{7}{l}{\textit{Lexicon Diversity}} \\
    Vocabulary size & 87.83 & \cellcolor{gray!25} 4.10 & -0.42 & -0.21 & -2.05 & 0.19 \\
    Thai words & 65.00 & -1.06 & -1.48 & 0.38 & -3.87 & 1.35 \\
    Long words & 4.07 & -1.05 & 0.51 & 0.09 & 1.10 & -0.32 \\
    Dictionary words & 71.71 & -4.85 & 0.45 & 1.36 & -3.20 & -1.06 \\
    &  & &\\
    \hline
    \multicolumn{7}{l}{\textit{Function Words}} \\
    Pronoun & 9.42 & -0.18 & \cellcolor{gray!25} -2.66 